In [ ]:
import pandas as pd
import json

In [ ]:
month = input("yymm")

#### Read bank data

In [ ]:
cols = {"Transaktionsdag": "date", "Referens": "ref", "Beskrivning": "info", "Belopp": "amount"}
dtypes = {"Beskrivning": "string", "Belopp": "float64"}

df = pd.read_csv(
    f"data/bank/{month}.csv",
    skiprows=1,
    usecols=cols.keys(),
    dtype=dtypes,
    parse_dates=["Transaktionsdag"],
).rename(columns=cols).iloc[::-1].reset_index(drop=True)

# drop transfer to savings account and to Revolut
df = df[(df["ref"] != "832797349214242") & (~df["ref"].str.contains("Revolut"))]

df.head()

#### Add Revolut data

In [ ]:
cols = {"Started Date": "date", "Description": "info", "Amount": "amount"}
dtypes = {"Description": "string", "Amount": "float64"}

df_revolut = pd.read_csv(
    f"data/revolut/{month}.csv",
    usecols=["Started Date", "Description", "Amount"],
    parse_dates=["Started Date"],
    date_parser=lambda x: pd.to_datetime(x).normalize(),
    dtype=dtypes,
).rename(columns=cols)
df_revolut.insert(1, "ref", "")

# drop top-ups
df_revolut = df_revolut[(df_revolut["info"] != "Card Top-Up")]

df = pd.concat([df, df_revolut], axis=0, ignore_index=True).sort_values("date").reset_index(drop=True)
df

#### Add categories

In [ ]:
df["category"] = pd.NA

with open("categories.json") as fp:
    category_map = json.load(fp)
    
for category, patterns in category_map.items():
    for p in patterns:
        df.loc[df["info"].str.contains(p, case=False), "category"] = category

df

In [ ]:
df[df["category"].isna()]

In [ ]:
import time
from IPython.display import clear_output

for idx, row in df[df["category"].isna()].iterrows():
    display(row.to_frame().T)
    category = input()
    while category := input() not in category_map.keys():
        print(f"Unknown category {category}")
        category = input()
    df.loc[idx, "category"] = category
    clear_output()

In [ ]:
df

#### Append to `history.csv`

In [ ]:
history_path = "data/history.csv"
history_df = pd.read_csv(history_path, parse_dates=["date"])

In [ ]:
if df["date"].isin(history_df["date"]).sum() > len(df) / 2:
    raise ValueError("The data might already be added to the history.")
else:
    history_df = pd.concat([history_df, df], axis=0, ignore_index=True).sort_values("date")
    history_df.to_csv(history_path, index=False)